In [1]:
import tensorflow as tf
import tensorflow.compat.v1 as tf_compat
from tensorflow.compat.v1.lite import TFLiteConverter

2023-04-12 13:38:30.562125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 13:38:30.685101: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-12 13:38:30.688894: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64${LD_LIBRARY_PATH:+:}
2023-04-12 13:38:30.688907: I tenso

In [2]:
tf_version = tf.version.VERSION[0] + tf.version.VERSION[2] + tf.version.VERSION[3]
tf_version

'211'

In [3]:
# Load the frozen model
# model_dir = '.'
# loaded_model = tf.saved_model.load('.')

# # Convert the image model
# custom_tags = set([''])
# converter_image = tf.lite.TFLiteConverter.from_saved_model(model_dir, tags=None) # path to the SavedModel directory
# quantize_string = ""
# # converter_image.optimizations, quantize_string = [tf.lite.Optimize.DEFAULT], "_dyq"
# converter_image.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# tflite_image_model = converter_image.convert()

# # Save the model
# tflite_model_path = "mobilefacenet" + quantize_string  + "_TF" + tf_version + ".tflite"
# with open(tflite_model_path, 'wb') as f:
#     f.write(tflite_image_model)

In [4]:
# Disable TensorFlow 2.x behavior
tf_compat.disable_v2_behavior()
tf_compat.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
# Load the frozen model
with tf_compat.gfile.GFile('./saved_model.pb', 'rb') as f:
    graph_def = tf_compat.GraphDef()
    graph_def.ParseFromString(f.read())

In [6]:
# for node in graph_def.node:
#     print(node.name, node.op)
#     if 'Placeholder' in node.op:
#         print('Input node name:', node.name)
#     elif 'Softmax' in node.op or 'Sigmoid' in node.op:
#         print('Output node name:', node.name)

In [7]:
# Convert the image model
input_node_name = "img_inputs"
output_node_name = "embeddings"

with tf_compat.Session() as sess:
    tf.import_graph_def(graph_def, name='')
    
    input_tensor = sess.graph.get_tensor_by_name(input_node_name + ':0')
    output_tensor = sess.graph.get_tensor_by_name(output_node_name + ':0')
    
    converter_image = TFLiteConverter.from_session(sess, [input_tensor], [output_tensor])
    converter_image.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    quantize_string = "_unq"
    # converter_image.optimizations, quantize_string = [tf.lite.Optimize.DEFAULT], "_dyq"
    tflite_image_model = converter_image.convert()

2023-04-12 13:38:32.465166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-12 13:38:32.465379: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64${LD_LIBRARY_PATH:+:}
2023-04-12 13:38:32.465437: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64${LD_LIBRARY_PATH:+:}
2023-04-12 13:38:32.465489: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.1

In [8]:
# Save the model
tflite_model_path = "mobilefacenet" + quantize_string + "_TF" + tf_version + ".tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_image_model)